## Knapsack Problem

### Problem Statement

We have a set of cars of different sizes and values and a parking lot at the dealership with a limited car capacity. How do we choose which cars to pack into the parking lot to maximize the total sale value?

### Define Variables and Parameters

* $N$ is the number of cars
* $s_i$ is the size of car $i$
* $v_i$ is the value of car $i$
* $x_i$ is the decision variables: $x_i = 1$ if car $i$ is chosen and $x_i = 0$ if not
* $c_i$ is the carbon tax of the car $i$
* $C$ is the total carbon tax amount the dealership paid. 
* $S$ is the capacity of the parking lot

### Identify Constraints and Make Assumptions

* Total area of the parked items is less than the parking lot capacity

In [1]:
# Import Packages
import numpy as np
import cvxpy as cp
import pandas as pd

In [6]:
# Data Loading
df = pd.read_csv('data/used_cars.csv')
print(list(df.columns))

['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']


In [7]:
# data cleaning
df = df.dropna()
df_new = df[df['fuel_type'] != "not supported"]
print(df_new['fuel_type'].unique())
df_new = df_new[df_new['fuel_type'] != '–']

df_new.drop(columns=["model_year", "milage","engine","transmission","ext_col","int_col","accident","clean_title"], inplace=True)

# Remove the dollar sign and convert to numeric
df_new['price'] = df_new['price'].str.replace(r'[$,]', '', regex=True).astype(float)
print(df_new.head())

['E85 Flex Fuel' 'Gasoline' 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–']
      brand                            model      fuel_type    price
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0
1   Hyundai                     Palisade SEL       Gasoline  38005.0
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0
7       BMW                           740 iL       Gasoline   7300.0


In [ ]:
df_new["Depreciation Rate"] = [
    0.5 if type == "Diesel" else
    0.65 if type == "Gasoline" else
    0.7 if type == 'E85 Flex Fuel' else
    0.80 if type == "Hybrid" else
    0.95
    for type in df_new["fuel_type"]
]
df_new["Carbon Tax"] = [
    500 if type == "Diesel" else
    200 if type == "Gasoline" else
    150 if type == 'E85 Flex Fuel' else
    120 if type == "Hybrid" else
    50
    for type in df_new["fuel_type"]
]
df_new["Buying Price"] = df_new["price"]*df_new["Depreciation Rate"] + df_new["Carbon Tax"]
df_new["Profit"] = df_new["price"]-df_new["Buying Price"]
# Create the "imported" column, default hybrid and plug-in hybrid rows to 0
df_new['imported'] = np.where(df_new['fuel_type'].isin(['Plug-In Hybrid', 'Hybrid']), 0, np.nan)
# Randomly assign 0 or 1 to the rest of the rows
df_new['imported'] = df_new['imported'].apply(lambda x: np.random.choice([0, 1]) if np.isnan(x) else x)
print(df_new.head())

      brand                            model      fuel_type    price  \
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0   
1   Hyundai                     Palisade SEL       Gasoline  38005.0   
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0   
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0   
7       BMW                           740 iL       Gasoline   7300.0   

   Depreciation Rate  Carbon Tax  Buying Price    Profit  imported  
0               0.70         150       7360.00   2940.00       0.0  
1               0.65         200      24903.25  13101.75       0.0  
3               0.80         120      12520.00   2980.00       0.0  
6               0.65         200      20350.00  10650.00       0.0  
7               0.65         200       4945.00   2355.00       0.0  


In [ ]:
# total number of cars 
N = len(df_new)
# value of cars varies
profit = df_new['Profit']
# size of the cars varies
s = np.random.randint(8,10,N)
# total size of all cars
size = np.sum(s)
# parking lot capacity
S = np.random.randint(size//2,2*size//3)

# Buying price for each car
b = df_new["Buying Price"]
# total amount of buying cost
budget = np.sum(b)
# total budget the dealership has
C = np.random.randint(budget//5,budget//4)

fee_electric = 300
fee_imported = 1000

# Outputs
print("Total Number of cars ready to sell: ", N)
print("Total car value: $",np.sum(profit))
print("Total Size of cars: ",size, "m^2")
print("Total buying cost amount: $", budget)
print("------------------------------------------")
print("Parking Lot Capacity: ",S, "m^2")
print("Available Budget Amount: $", C)


print("---------------------------------------------------------------")
print("DealershipDecisionOptimizerSystem(DDOS) is working....")
# Solution
x = cp.Variable(N,integer=True)
# Penalty fees
electric_penalty = fee_electric * cp.max(cp.multiply(x,df_new['fuel_type'].apply(lambda g: g == 'Plug-In Hybrid').astype(int)))
imported_penalty = fee_imported * cp.max(cp.multiply(x,df_new['Imported']))
obj = cp.Maximize(cp.sum(cp.multiply(profit,x))-electric_penalty-imported_penalty)
constraints = [cp.sum(cp.multiply(s,x)) <= S,
               cp.sum(cp.multiply(b,x)) <= C,
               
                 x <= 1, x >= 0]
prob = cp.Problem(obj,constraints)
prob.solve()
print("---------------------------------------------------------------")
print(np.sum(x.value), "cars sold")
print("Profit: $", profit@x.value)
print("Actual Size of cars sold: ", s@x.value, "m^2")
print("Actual budget spent: $", b@x.value)

Total Number of cars ready to sell:  3223
Total car value: $ 45042160.949999996
Total Size of cars:  27401 m^2
Total buying cost amount: $ 88213516.05000001
------------------------------------------
Parking Lot Capacity:  16393 m^2
Available Budget Amount: $ 19545090
---------------------------------------------------------------
DealershipDecisionOptimizerSystem(DDOS) is working....
---------------------------------------------------------------
176.0 cars sold
Profit: $ 11508878.55
Actual Size of cars sold:  1492.0 m^2
Actual budget spent: $ 19544909.45


In [ ]:
# approximation with greedy algorithm
fuel = df_new["fuel_type"]
imported = df_new["imported"]

value_per_size = profit/s
print(value_per_size.round(2))

items = []
counter_import = 0
counter_EV = 0

EV_fee = 500
license_fee = 700

for _ in range(N):
    i = np.argmax(value_per_size)
    if (((fuel[i]=="Hybrid")or(fuel[i]=="Plug-In Hybrid")) and counter_EV==0):
        value = profit[i]+EV_fee
        vps = value/s[i]
        if (vps > )

    items.append(i)
    value_per_size[i] = 0.
    size = sum([s[k] for k in items])
    carbon = sum([b[k]for k in items])
    if((size > S) or (carbon > C) ):
        items = items[:-1]
        break

sorted(items)

In [ ]:
# Initialize variables
fuel = df_new['fuel_type'].values
imported = df_new['imported'].values
profit = df_new['profit'].values
size = df_new['size'].values
budget = df_new['budget'].values

value_per_size = profit / size
items = []

counter_import = 0
counter_EV = 0
EV_fee = 200
license_fee = 400
should_break=False

# check if i-th car is 1st EV about to put into bin
def check_EV(i):
    adjusted_profit = profit[i]
    if ((fuel[i] == 'Hybrid') or (fuel[i] == 'Plug-In Hybrid')) and counter_EV == 0:
        adjusted_profit -= EV_fee
        counter_EV=1
        return True
        
# check if i-th car is 1st imported car about to put into bin
def check_import(i):
    adjusted_profit = profit[i]
    if imported[i] == 1 and counter_import == 0:
        adjusted_profit -= license_fee
        counter_import = 1
        return True
    else:
        return False
    
def compare(i):
    # branch on EV case or import case:
    if (check_EV(i)):
    return 1
    
def check_constraints():
    if total_size > S or total_budget > C:
        items.pop()
        return True
    else:
        return False



while True:
    # Find the most valuable item
    i = np.argmax(value_per_size)
    if value_per_size[i] == 0:
        break

    # check if we need to enter the compare hassle:
    if (check_EV(i) or check_import(i)):
        compare(i)
    else:
        items.append(i)
        value_per_size[i] = 0
        total_size = sum([size[k] for k in items])
        total_budget = sum([carbon[k] for k in items])
        if(check_constraints()):
            should_break=True
    if (should_break):
        break










    # Check the next best option
    value_per_size[i] = 0
    j = np.argmax(value_per_size)
    next_adjusted_profit = profit[j]
    if ((fuel[j] == 'Hybrid') or (fuel[j] == 'Plug-In Hybrid')) and counter_EV == 0:
        next_adjusted_profit -= EV_fee
        counter_EV = 1
    if imported[j] == 1 and counter_import == 0:
        next_adjusted_profit -= license_fee
        counter_import = 1

    # Compare profits
    if (adjusted_profit/s[i]) >= (next_adjusted_profit/s[j]):
        # Add the current item
        items.append(i)
        value_per_size[i] = 0
        total_size = sum([size[k] for k in items])
        total_carbon = sum([carbon[k] for k in items])
        # check on counters:
        if 
    else:
        items.append(j)
        value_per_size[j] = 0
        total_size = sum([size[k] for k in items])
        total_carbon = sum([carbon[k] for k in items])
        #restore the value_per_size in index i
        value_per_size[i] = profit[i]/size[i]

    # Update counters
    if (fuel[i] == 'Hybrid' or fuel[i] == 'Plug-In Hybrid') and counter_EV == 0:
        counter_EV += 1
    if imported[i] == 1 and counter_import == 0:
        counter_import += 1

    # Check capacity constraints
    if total_size > S or total_carbon > C:
        items.pop()
        break
    else:
        # Restore value_per_size for the current item if skipped
        value_per_size[i] = adjusted_profit / size[i]